# Projeto Criação de um ML para detecção de fraudes

<h3>Etapa 1</h3>

<h4>1.0 - Preparando o ambiente</h4>

<h4>1.1 - Importando bibliotecas e banco de dados</h4>

<h4>1.2 - Estatística do banco de dados</h4>

<h4>1.3 - Limpando os dados</h4>

<h4>1.4 - Pipeline em ciência de dados</h4>

<h4>1.5 - Levantando hipóteses</h4>



<h4>1.1 - Importando bibliotecas e banco de dados</h4>

In [10]:
# Bibiotecas utilizadas para carregamento e manipulação dos dados
import numpy as np
import pandas as pd

from IPython.display import display

In [6]:
fraude_df = pd.read_csv("../Fraud_Data.csv",header=0)
ISP_df = pd.read_csv("../IpAddress_to_Country.csv",header=0)

In [3]:
fraude_df.dtypes

id                  int64
cadastro           object
compra             object
valor               int64
id_dispositivo     object
fonte              object
browser            object
genero             object
idade               int64
ip                float64
fraude              int64
dtype: object

In [25]:
print('Checando valores nulos')
for col in fraude_df.columns:
    print('# de valores nulos em {}: {}'.format(col,fraude_df[col].isnull().sum()))

Checando valores nulos
# de valores nulos em id: 0
# de valores nulos em cadastro: 0
# de valores nulos em compra: 0
# de valores nulos em valor: 0
# de valores nulos em id_dispositivo: 0
# de valores nulos em fonte: 0
# de valores nulos em browser: 0
# de valores nulos em genero: 0
# de valores nulos em idade: 0
# de valores nulos em ip: 0
# de valores nulos em fraude: 0


<h4>1.2 - Estatística do banco de dados</h4>

In [5]:
fraude_df.describe()

,id,valor,idade,ip,fraude
count,151112.000000,151112.000000,151112.000000,1.511120e+05,151112.000000
mean,200171.040970,36.935372,33.140704,2.152145e+09,0.093646
std,115369.285024,18.322762,8.617733,1.248497e+09,0.291336
min,2.000000,9.000000,18.000000,5.209350e+04,0.000000
25%,100642.500000,22.000000,27.000000,1.085934e+09,0.000000
50%,199958.000000,35.000000,33.000000,2.154770e+09,0.000000
75%,300054.000000,49.000000,39.000000,3.243258e+09,0.000000
max,400000.000000,154.000000,76.000000,4.294850e+09,1.000000


In [6]:
for col in ISP_df.columns:
    print('{}:{}'.format(col,ISP_df[col].dtype))

limite_inferior_ip:float64
limite_superior_ip:int64
pais:object


In [7]:
ip_floats = ISP_df.limite_inferior_ip.apply(lambda x: not x.is_integer()).sum()
print('Número de limites inferiores de endereço IP com decimal não-zero: {}'.format(ip_floats))

Número de limites inferiores de endereço IP com decimal não-zero: 0


In [8]:
#ISP_df.limite_inferior_ip = ISP_df.limite_inferior_ip.astype(int)

In [9]:
for col in ISP_df.columns:
    print('# de valores nulos em {}: {}'.format(col,ISP_df[col].isnull().sum()))

# de valores nulos em limite_inferior_ip: 0
# de valores nulos em limite_superior_ip: 0
# de valores nulos em pais: 0


In [10]:
ISP_df.describe()

,limite_inferior_ip,limite_superior_ip
count,1.388460e+05,1.388460e+05
mean,2.724532e+09,2.724557e+09
std,8.975215e+08,8.974979e+08
min,1.677722e+07,1.677747e+07
25%,1.919930e+09,1.920008e+09
50%,3.230887e+09,3.230888e+09
75%,3.350465e+09,3.350466e+09
max,3.758096e+09,3.758096e+09


In [11]:
# Analisando se coincide o máximo do limite inferior com o limite superior 
display(ISP_df[ISP_df.limite_inferior_ip == ISP_df.limite_inferior_ip.max()])
display(ISP_df[ISP_df.limite_superior_ip == ISP_df.limite_superior_ip.max()])

,limite_inferior_ip,limite_superior_ip,pais
138845,3758096128,3758096383,Australia


,limite_inferior_ip,limite_superior_ip,pais
138845,3758096128,3758096383,Australia


In [12]:
ISP_df.describe()

,limite_inferior_ip,limite_superior_ip
count,1.388460e+05,1.388460e+05
mean,2.724532e+09,2.724557e+09
std,8.975215e+08,8.974979e+08
min,1.677722e+07,1.677747e+07
25%,1.919930e+09,1.920008e+09
50%,3.230887e+09,3.230888e+09
75%,3.350465e+09,3.350466e+09
max,3.758096e+09,3.758096e+09


In [13]:
fraude_df.describe()

,id,valor,idade,ip,fraude
count,151112.000000,151112.000000,151112.000000,1.511120e+05,151112.000000
mean,200171.040970,36.935372,33.140704,2.152145e+09,0.093646
std,115369.285024,18.322762,8.617733,1.248497e+09,0.291336
min,2.000000,9.000000,18.000000,5.209350e+04,0.000000
25%,100642.500000,22.000000,27.000000,1.085934e+09,0.000000
50%,199958.000000,35.000000,33.000000,2.154770e+09,0.000000
75%,300054.000000,49.000000,39.000000,3.243258e+09,0.000000
max,400000.000000,154.000000,76.000000,4.294850e+09,1.000000


In [28]:
isp = fraude_df.ip[13]

pais = ISP_df[(ISP_df.limite_inferior_ip <= isp) & (ISP_df.limite_superior_ip >= isp)].pais
pais.shape[0],isp

(1, 3178510014.63508)

In [22]:
ISP_df.limite_inferior_ip[0]

16777216.0

In [ ]:
# Criando a variável pais que vai fazer uma contagem das seguintes condições, 
# ela vai ler o dataframe ISP_df e a ideia é que o argumento tem que estar entre o LI_ip e LS_ip, 
# pois isso significa que ele está dentro da lista de países que temos no dataframe ISP_df
def isp_counts(isp):
    pais = ISP_df[(ISP_df.limite_inferior_ip <= isp) & (ISP_df.limite_superior_ip >= isp)].pais
    if pais.shape[0] == 1:
        return print("Único")
    elif pais.shape[0] > 1:
        return print("Vários países")
    elif pais.shape[0] == 0:
        if isp > ISP_df.limite_superior_ip.max():
            return print("Muito grande")
        elif isp < ISP_df.limite_inferior_ip.min():
            return print("Muito pequeno")
        else:
            return print("Faltando")
        
ip_counts = fraude_df.ip.apply(isp_counts)
ip_counts.value_counts()

In [83]:
# Criar a coluna pais 
paises = []
for i in range(len(fraude_df)):
    ip_address = fraude_df.loc[i, 'ip']
    tmp = ISP_df[(ISP_df['limite_inferior_ip'] <= ip_address) &
                          (ISP_df['limite_superior_ip'] >= ip_address)]
    if len(tmp) == 1:
        paises.append(tmp['pais'].values[0])
    else:
        paises.append('ausente')

fraude_df['pais'] = paises
fraude_df.head()

,id,cadastro,compra,valor,id_dispositivo,fonte,browser,genero,idade,ip,fraude,pais
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,ausente
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States


In [ ]:
# Outra forma de criar a coluna pais, é por esta função
def ip_para_pais(ip):
    tmp = ISP_df[(ISP_df.limite_inferior_ip <= ip)
                & (ISP_df.limite_superior_ip >= ip)].pais
    if tmp.shape[0] == 1:
        return tmp.iloc[0]
    else tmp.shape[0] != 1:
        return 'ausente'
        
fraude_df["pais"] = fraude_df.ip.apply(ip_para_pais)

In [56]:
fraude_df.pais.value_counts(normalize=True)

United States     0.384146
ausente           0.145362
China             0.079663
Japan             0.048348
United Kingdom    0.029713
                    ...   
Turkmenistan      0.000007
Yemen             0.000007
Madagascar        0.000007
South Sudan       0.000007
Guadeloupe        0.000007
Name: pais, Length: 182, dtype: float64

In [84]:
#fraude_df.to_csv("Dados_fraude_com_pais", index=False)
fraude_df = pd.read_csv("Dados_fraude_com_pais", header=0)
fraude_df.head()

,id,cadastro,compra,valor,id_dispositivo,fonte,browser,genero,idade,ip,fraude,pais
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NaN
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States


In [90]:
paises

['Japan',
 'United States',
 'United States',
 'ausente',
 'United States',
 'Canada',
 'ausente',
 'United States',
 'China',
 'United States',
 'United States',
 'ausente',
 'ausente',
 'Brazil',
 'ausente',
 'India',
 'United States',
 'ausente',
 'Argentina',
 'United Kingdom',
 'ausente',
 'ausente',
 'China',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'South Africa',
 'United States',
 'United States',
 'China',
 'Japan',
 'United States',
 'United States',
 'Germany',
 'United States',
 'China',
 'China',
 'ausente',
 'Japan',
 'ausente',
 'Mexico',
 'United States',
 'Brazil',
 'United States',
 'United States',
 'Sweden',
 'ausente',
 'Korea Republic of',
 'Brazil',
 'Peru',
 'United States',
 'United States',
 'Portugal',
 'Bangladesh',
 'Korea Republic of',
 'France',
 'United States',
 'France',
 'ausente',
 'Taiwan; Republic of China (ROC)',
 'Russian Federation',
 'ausente',
 'United States',
 'Belgium',
 'Germany',
 'United

In [92]:
fraude_df[fraude_df.pais=='Brazil']

,id,cadastro,compra,valor,id_dispositivo,fonte,browser,genero,idade,ip,fraude,pais
13,299320,2015-03-03 19:17:07,2015-04-05 12:32:36,50,RMKQNVEWGTWPC,Direct,Safari,M,38,3.178510e+09,0,Brazil
44,132826,2015-07-25 10:41:51,2015-08-26 05:40:36,18,UOSPQFPVALJAX,SEO,FireFox,F,35,3.364603e+09,0,Brazil
50,254562,2015-05-12 18:58:01,2015-07-31 11:36:16,75,VYSNKKTTQIXQC,Ads,FireFox,F,45,2.977122e+09,0,Brazil
96,244618,2015-06-11 12:40:52,2015-08-30 08:09:07,81,MSPMJUTDCXZSC,SEO,FireFox,M,20,3.014511e+09,0,Brazil
127,235741,2015-07-07 03:33:01,2015-07-12 05:34:50,63,EWMIAJTNJQYWM,Ads,FireFox,F,19,3.208200e+09,0,Brazil
...,...,...,...,...,...,...,...,...,...,...,...,...
150765,284519,2015-05-21 04:53:45,2015-06-07 23:25:26,42,ZPAHZRZICKMLW,Ads,Chrome,M,43,3.174104e+09,0,Brazil
150833,152317,2015-05-29 21:51:41,2015-08-13 03:00:08,54,BCCOSLIYKMMJD,Ads,Safari,M,20,2.402729e+09,0,Brazil
150965,181331,2015-03-12 23:54:19,2015-04-18 04:48:18,53,FNYLMGNUZJGPN,Direct,FireFox,F,27,3.018166e+09,0,Brazil
150994,222304,2015-04-07 05:06:26,2015-04-17 16:55:27,38,WNNJKXVJGXBRI,Ads,IE,M,22,3.178845e+09,1,Brazil
